In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
def get_nav_link():
  response = requests.get('https://www.cnnindonesia.com')

  # Untuk mengecek respon url dan mencari menu sidebar
  if response.status_code == 200:
    # Get Nav
    html = response.content
    nav = BeautifulSoup(html, 'html.parser').find_all('a', class_='navbar__item')
    nav_link = {i.text.strip() : i.get('href') for i in nav if i.text.strip() not in ['Home']}

    # Update Nav Link
    for category, link in nav_link.items():
      nav_content = requests.get(link).content
      new_link = BeautifulSoup(nav_content, 'html.parser').find('a', {'dtr-act' : 'button indeks berita'}).get('href')
      nav_link[category] = new_link

    return nav_link

  else:
    print(f'Response https://www.cnnindonesia.com : {response.status_code}')

In [3]:
def get_data(url):
  print(url)
  response = requests.get(url)
  if response.status_code == 200:
    data = []
    html = response.content
    content = BeautifulSoup(html, 'html.parser')
    judul = content.find('h1', class_='mb-2 text-[28px] leading-9 text-cnn_black').text.strip()
    date = content.find('div', class_='text-cnn_grey text-sm mb-4').text.strip()
    category = content.find_all('a', {'dtr-evt' : 'breadcrumb'})[1].text.strip()
    article = content.find('div', class_='detail-text text-cnn_black text-sm grow min-w-0').find_all('p')
    text = ''
    for i in article:
      if i.text.strip() not in ['SCROLL TO RESUME CONTENT', 'ADVERTISEMENT']:
        text += i.text.strip() + ' '
    data.append(url)
    data.append(judul)
    data.append(date)
    data.append(text.replace('\xa0', ''))
    data.append(category.strip())
    return data

In [4]:
def get_content(url, len_data, target):
  data = []
  response = requests.get(url)
  if response.status_code == 200:
    html = response.content
    container = BeautifulSoup(html, 'html.parser')
    list_content = container.find('div', class_='flex flex-col gap-5').find_all('article', class_='flex-grow')
    next_button = container.find('a', {'dtr-act' : 'halaman selanjutnya'}).get('href')
    idx = 0

    while idx != len(list_content):
      subjudul = list_content[idx].find('span', class_='text-cnn_red block subjudul mb-1')
      if len(list_content[idx].find('h2').text.strip().split(':')) < 2 and subjudul == None:
        if len_data != target:
          link_content = list_content[idx].find('a').get('href')
          data_berita = get_data(link_content)
          data.append(data_berita)
          len_data += 1
          print(data_berita)
        else:
          break
          return (data, next_button)

      idx += 1

  else:
    print(f'Response {url} : {response.status_code}')
  return (data, next_button)


In [5]:
def scraping_cnn(category=['Nasional'], num_data=2):
  category_link = get_nav_link()
  data = []

  for i in category:
    len_data_category = 0
    while len_data_category != num_data:
      new_data, next_button = get_content(category_link[i], len_data_category, num_data)
      data += new_data
      len_data_category += len(new_data)
      category_link[i] = next_button

  print(f'Berhasil Mengambil data sebanyak {len(data)} berita dengan {len(category)} Category')

  df = pd.DataFrame(data, columns=['Url', 'Judul', 'Tanggal', 'Artikel', 'Category'])
  df.to_csv('Data Berita CNN.csv', index=False)

In [6]:
scraping_cnn(category=['Teknologi', 'Ekonomi', 'Olahraga'], num_data=150)

https://www.cnnindonesia.com/teknologi/20231108154142-206-1021591/bakal-jadi-saingan-galaxy-fold-5-apa-keunggulan-oppo-find-n3
['https://www.cnnindonesia.com/teknologi/20231108154142-206-1021591/bakal-jadi-saingan-galaxy-fold-5-apa-keunggulan-oppo-find-n3', 'Bakal Jadi Saingan Galaxy Fold 5, Apa Keunggulan Oppo Find N3?', 'Rabu, 08 Nov 2023 18:31 WIB', 'Pabrikan smartphoneasal China, Oppo segera merilis Hp foldable pertamanya di RI, Oppo Find N3 Fold. Hp ini bakal menjadi pesaing utama Samsung Galaxy Fold 5, lalu apa keunggulanOppo Find N3 Fold? Baskoro Adiwiyono, Head of Public Relations Oppo Indonesia mengatakan salah satu keunggulan Find N3 Fold adalah Hp ini dibekali dengan kamera flagship. Oppo menggandeng manufaktur kamera, Hassleblad untuk menggarap kamera belakang Find N3 Fold. "Kami luncurkan dengan fitur flagship yang sesungguhnya. Kenapa fitur flagship yg sesungguhnya? Karena kameranya flagship," kata Baskoro saat ditemui di Jakarta Selatan, Rabu (8/11). SCROLL TO CONTINUE W